# 🚢 AIS航跡 + ERA5気象データ付加テンプレート（Binder対応）

このノートブックは **Google Colab不要**、**ブラウザ上のBinder環境**で実行できます。

ERA5の2m気温、風、降水、放射データをAIS位置情報に補間して付加します。

In [ ]:
!pip install cdsapi pandas openpyxl numpy xarray netCDF4 tqdm

In [ ]:
import pandas as pd
import cdsapi, xarray as xr, numpy as np
from datetime import datetime
from tqdm import tqdm
import os
from IPython.display import FileLink

print('✅ ライブラリ読み込み完了')

## 🔑 Step 1: CDS APIキー設定
Copernicusのアカウントページで取得した **APIキー** と **URL** を入力してください。

In [ ]:
os.environ['CDSAPI_URL'] = 'https://cds.climate.copernicus.eu/api'
os.environ['CDSAPI_KEY'] = input('CDS APIキーを入力してください：')

c = cdsapi.Client()

## 📂 Step 2: AISファイルをアップロード
Excel（.xlsx）ファイルをアップロードしてください。列名に「lat」「lon」「time」または「date」を含む列が必要です。

In [ ]:
from google.colab import files  # Binderでも使用可能
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

df.rename(columns=lambda x: x.strip().lower(), inplace=True)
lat_col = [c for c in df.columns if 'lat' in c][0]
lon_col = [c for c in df.columns if 'lon' in c or 'lng' in c][0]
time_col = [c for c in df.columns if 'time' in c or 'date' in c][0]
df[time_col] = pd.to_datetime(df[time_col])

print('✅ データ読み込み完了')

## 🌍 Step 3: 範囲と期間の設定

In [ ]:
min_lat, max_lat = df[lat_col].min()-1, df[lat_col].max()+1
min_lon, max_lon = df[lon_col].min()-1, df[lon_col].max()+1

start_date = df[time_col].min().strftime('%Y-%m-%d')
end_date = df[time_col].max().strftime('%Y-%m-%d')

print(f'📍範囲: {min_lat}–{max_lat}N, {min_lon}–{max_lon}E')
print(f'🕒期間: {start_date}〜{end_date}')

## 🌦 Step 4: ERA5データを取得

In [ ]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            '2m_temperature',
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
            'total_precipitation',
            'surface_solar_radiation_downwards'
        ],
        'year': list({d.year for d in df[time_col]}),
        'month': list({d.month for d in df[time_col]}),
        'day': list({d.day for d in df[time_col]}),
        'time': [f"{h:02d}:00" for h in range(24)],
        'format': 'netcdf',
        'area': [max_lat, min_lon, min_lat, max_lon],
    },
    'era5_subset.nc'
)

print('✅ ERA5データ取得完了')

## 📊 Step 5: データ補間と追加

In [ ]:
ds = xr.open_dataset('era5_subset.nc')

def interp_var(var):
    return ds[var].interp(latitude=('points', df[lat_col]), longitude=('points', df[lon_col]), time=('points', df[time_col]))

df['air_temp_C'] = interp_var('t2m').values - 273.15
df['wind_u'] = interp_var('u10').values
df['wind_v'] = interp_var('v10').values
df['precip_mm'] = interp_var('tp').values * 1000
df['solar_Jm2'] = interp_var('ssrd').values

df['wind_speed_mps'] = np.sqrt(df['wind_u']**2 + df['wind_v']**2)
df['wind_dir_deg'] = (180/np.pi) * np.arctan2(df['wind_u'], df['wind_v']) + 180

print('✅ データ補間完了')

## 💾 Step 6: 出力

In [ ]:
output_filename = 'AIS_with_weather.xlsx'
df.to_excel(output_filename, index=False)

FileLink(output_filename)